In [41]:
import pandas as pd
import numpy as np

In [3]:
from openskill.models import PlackettLuce, BradleyTerryFull

targetElo = 1000
baseElo = 500
targetSeason = 'f24'
model = PlackettLuce(beta=25.0/120.0)

In [53]:
df_races = pd.read_json("s24teamraces.json")
df_sailor_info = pd.read_json("sailor_data2.json")
df_sailorTRinfo = pd.read_json('trSailorInfo.json')
df_races

,raceID,raceNum,round,teamAName,teamAUni,teamANick,teamALink,teamAID,teamABoats,teamAScore,teamAOutcome,teamBName,teamBUni,teamBNick,teamBLink,teamBID,teamBBoats,teamBScore,teamBOutcome
0,s24/icsa-open-team-racing-national/1,1,Round of 16,Yale,Yale University,Bulldogs,yale,73983,"[{'skipperName': 'Jack Egan '25', 'skipperLink...",2-3-4,win,Dartmouth,Dartmouth College,Big Green,dartmouth,73966,"[{'skipperName': 'Robert Bragg '23', 'skipperL...",1-5-6,lose
1,s24/icsa-open-team-racing-national/2,2,Round of 16,Harvard,Harvard University,Crimson,harvard,73968,"[{'skipperName': 'Justin Callahan '26', 'skipp...",1-2-5,win,Coast Guard,U. S. Coast Guard Academy,Bears,coast-guard,73982,"[{'skipperName': 'Colman Schofield '25', 'skip...",3-4-6,lose
2,s24/icsa-open-team-racing-national/3,3,Round of 16,Navy,U. S. Naval Academy,Midshipmen,navy,73958,"[{'skipperName': 'Jack Welburn '26', 'skipperL...",1-2-3,win,Wisconsin,University of Wisconsin,Badgers,wisconsin,73959,"[{'skipperName': 'Gavin Dempsey '26', 'skipper...",4-5-6,lose
3,s24/icsa-open-team-racing-national/4,4,Round of 16,MIT,Massachusetts Institute of Technology,Engineers,mit,73972,"[{'skipperName': 'Sam Bruce '24', 'skipperLink...",1-2-3,win,Tufts,Tufts University,Jumbos,tufts,73980,"[{'skipperName': 'Ben Mueller '26', 'skipperLi...",4-5-6,lose
4,s24/icsa-open-team-racing-national/5,5,Round of 16,Brown,Brown University,Bears,brown,73964,"[{'skipperName': 'Guthrie Braun '26', 'skipper...",4-5-6,lose,Roger Williams,Roger Williams University,Hawks,roger-williams,73979,"[{'skipperName': 'Aidan Hoogland '24', 'skippe...",1-2-3,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,s24/mustang-team-race/6,6,Round 1,Southern Cal,University of Southern California,Trojans,southern-cal,71720,"[{'skipperName': 'Luke Harris '27', 'skipperLi...",3-4-5,lose,Cal Maritime,California Maritime Academy,Keelhaulers,cal-maritime,71717,"[{'skipperName': 'Clay Myers '25', 'skipperLin...",1-2-6,win
3174,s24/mustang-team-race/7,7,Round 1,UC Los Angeles,University of California at Los Angeles,Bruins,uc-los-angeles,71718,"[{'skipperName': 'Ian Wells '24', 'skipperLink...",2-4-6,lose,Cal Poly,Cal Poly University S.L.O.,Mustangs,cal-poly,71716,"[{'skipperName': 'Max Case '24', 'skipperLink'...",1-3-5,win
3175,s24/mustang-team-race/8,8,Round 1,UC Santa Cruz,University of California at Santa Cruz,Banana Slugs,uc-santa-cruz,71719,"[{'skipperName': 'Blake Roberts '25', 'skipper...",3-4-5,lose,Southern Cal,University of Southern California,Trojans,southern-cal,71720,"[{'skipperName': 'Luke Harris '27', 'skipperLi...",1-2-6,win
3176,s24/mustang-team-race/9,9,Round 1,Cal Poly,Cal Poly University S.L.O.,Mustangs,cal-poly,71716,"[{'skipperName': 'Max Case '24', 'skipperLink'...",3-5-6,lose,Cal Maritime,California Maritime Academy,Keelhaulers,cal-maritime,71717,"[{'skipperName': 'Clay Myers '25', 'skipperLin...",1-2-4,win


In [54]:
class Sailor:
    def __init__(self, name, key, year, links, teams=[], seasons=[], skipperRank=0, crewRank=0,skipperRating=baseElo, crewRating=baseElo, races=[], gender=""):
        self.name = name
        self.key = key
        self.gender = gender
        self.year = year
        self.links = links
        self.teams = teams
        self.races = []
        
        self.wtsr = model.rating(name=name)
        self.wtcr = model.rating(name=name)
        self.tsr = model.rating(name=name)
        self.tcr = model.rating(name=name)

In [65]:
def createPeople():
    people = {}
    
    for _,person in df_sailorTRinfo.iterrows():
        gender = ''
        key = person['key']
        if key in list(df_sailor_info['link']):
            data = df_sailor_info.loc[df_sailor_info['link'] == key]
            print(data)
            gender = data['gender'].iat[0]
        
        people[key] = Sailor(person['name'], 
                                       key, 
                                       person['year'],
                                       [person['link']], gender=gender)
    
    return people

In [66]:
people = createPeople()
for _,raceRow in df_races.iterrows():
    # print(raceRow)
    for type in ['Skipper', 'Crew']:
        
        womens = False
        
        teamAName = raceRow['teamAName']
        teamAKeys = [boat[type.lower() + 'Key'] if boat[type.lower() + 'Key'] is not None else 'Unknown' for boat in raceRow['teamABoats']] # the sailor keys
        teamANames = [boat[type.lower() + 'Name'] for boat in raceRow['teamABoats']] # the sailor keys
        teamARacers = [people[key] for key in teamAKeys if 'Unknown' not in key]
        
        teamBName = raceRow['teamBName']
        teamBKeys = [boat[type.lower() + 'Key'] if boat[type.lower() + 'Key'] is not None else 'Unknown' for boat in raceRow['teamBBoats']] # the sailor keys
        teamBNames = [boat[type.lower() + 'Name'] for boat in raceRow['teamBBoats']] # the sailor keys
        teamBRacers = [people[key] for key in teamBKeys if 'Unknown' not in key]
        
        genders = [p.gender for p in teamARacers + teamBRacers]
        womenCount = sum([1 if g == "F" else 0 for g in genders])
        womens = 'M' not in genders and womenCount >= 4
        
        teamARatings = []
        if womens:
            teamARatings = [r.tsr if type == 'Skipper' else r.tcr for r in teamARacers]
        else:
            teamARatings = [r.wtsr if type == 'Skipper' else r.wtcr for r in teamARacers]
            
        teamBRatings = []
        if womens:
            teamBRatings = [r.wtsr if type == 'Skipper' else r.wtcr for r in teamBRacers]
        else:
            teamBRatings = [r.tsr if type == 'Skipper' else r.tcr for r in teamBRacers]
            
        
        if len(teamARatings) < 1 or len(teamBRatings) < 1:
            print("not enough sailors in this race, skipping")
            continue
        
        ratings = model.rate([teamARatings, teamBRatings], 
                             [1 if raceRow['teamAOutcome'] == 'win' else 2, 1 if raceRow['teamBOutcome'] == 'win' else 2])
        
        for team, name in zip([teamARacers, teamBRacers], [teamAName, teamBName]):
            for racer, new_rating in zip(team, ratings[0]):
                racer.teams = [name]
                if type == 'Skipper':
                    if womens:
                        racer.wtsr = new_rating
                    else: 
                        racer.tsr = new_rating
                else:
                    if womens:
                        racer.wtcr = new_rating
                    else:
                        racer.tcr = new_rating
        
        # print(ratings)
    # break

           link       name first_name last_name gender    year school  \
8034  jack-egan  Jack Egan       Jack      Egan      M  2025.0   yale   

             id  external_id  
8034  3126964.0          NaN  
             link        name first_name last_name gender    year school  \
10865  elle-sykes  Elle Sykes       Elle     Sykes      F  2027.0   yale   

              id  external_id  
10865  3132301.0          NaN  
                  link             name first_name last_name gender    year  \
11110  morgan-pinckney  Morgan Pinckney     Morgan  Pinckney      M  2027.0   

      school         id  external_id  
11110   yale  3132550.0          NaN  
                  link              name first_name   last_name gender  \
8328  jessi-avila-shah  Jessi Avila-Shah      Jessi  Avila-Shah      F   

        year school         id  external_id  
8328  2025.0   yale  3126677.0          NaN  
                link            name first_name last_name gender    year  \
6366  teddy-nicolosi

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
allRows = []
for sailor,p in people.items():
    
    allRows.append([p.name, sailor, p.tsr.ordinal(target=targetElo, alpha=200 / model.sigma),p.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma), p.tcr.ordinal(target=targetElo, alpha=200 / model.sigma), p.teams, p.year, p.links, p.tsr.mu, p.tcr.mu])

df_fullsailors = pd.DataFrame(allRows, columns=['Sailor', 'key', 'SkipperOrdinal', 'WomenSkipperOrdinal', 'CrewOrdinal', 'Teams', 'GradYear', 'Links', 'SkipperMU','CrewMU'])
df_fullsailors

,Sailor,key,SkipperOrdinal,CrewOrdinal,Teams,GradYear,Links,SkipperMU,CrewMU
0,Jack Egan,jack-egan,1667.453374,1000.000000,[Yale],25,[jack-egan],37.149513,25.000000
1,Elle Sykes,elle-sykes,1000.000000,1650.387265,[Yale],27,[elle-sykes],25.000000,38.366160
2,Morgan Pinckney,morgan-pinckney,1539.861752,1000.000000,[Yale],27,[morgan-pinckney],33.527903,25.000000
3,Jessi Avila-Shah,jessi-avila-shah,1000.000000,1515.511735,[Yale],25,[jessi-avila-shah],25.000000,33.445934
4,Teddy Nicolosi,teddy-nicolosi,1680.731790,1000.000000,[Yale],24,[teddy-nicolosi],38.038405,25.000000
...,...,...,...,...,...,...,...,...,...
1093,Jack Blackman,jack-blackman,994.723315,1000.000000,[Southern Cal],24,[jack-blackman],23.388668,25.000000
1094,Genaro Flores - Machorro,genaro-flores-machorro,1000.000000,1006.644882,[Southern Cal],25,[genaro-flores-machorro],25.000000,23.914533
1095,Maggie McEachen,maggie-mceachen,1000.000000,882.795631,[UC Santa Cruz],27,[maggie-mceachen],25.000000,18.729955
1096,Aki Schuessler-Langeheine,aki-schuessler-langeheine,1000.000000,997.933483,[UC Los Angeles],24,[aki-schuessler-langeheine],25.000000,24.200112
